<a href="https://colab.research.google.com/github/Mxtsxw/PATFinder/blob/main/PATScrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
from googlesearch import search
import pandas as pd
import datetime

# URL de la page à analyser
url = "https://france-pat.fr/liste-des-pats/"

In [ ]:
def extract_PAT_links(url = "https://france-pat.fr/liste-des-pats/"):
  response = requests.get(url)
  # Vérifier si la requête a réussi
  if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    liste = soup.find(class_="search-list-container mt-2")
    liste = liste.find(class_="grid").find_all(class_="col-span-1")
    titles = [l.find(class_="block-list-pat").find("h4").get_text() for l in liste]
    return titles

In [ ]:
def get_links_from_search_results(query, stop=5):
  search_url = f"https://www.google.com/search?q={query}"

  return [res for res in search(search_url, lang="fr", stop=4, safe=None)]

In [ ]:
def contains_keywords(url, keyword):
  try:
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        keyword_lower = [word.lower() for word in keyword]
        text_elements = soup.find_all(string=True)
        for element in text_elements:
            words = element.strip().lower().split()
            if any(word in keyword_lower for word in words):
                return True
        return False
  except:
    print(f"Failed to retieve {url}")
    return False

In [ ]:
# Initialization
# keywords_veg = ["végétal", "végétale", "végétales", "végétarien", "végétariens", "végétariennes"]
keywords = ["légumineuse", "légumineuses"]

In [ ]:
# Step 1: Retrieve all PATs
PATs = extract_PAT_links()
res = []

# Step 2: Retrieve first results
for i in range(len(PATs[:100])):
    pat = PATs[i]
    entry = []
    links = get_links_from_search_results(pat)
    contained_links = []
    keyword_found = False
    for link in links:
        if contains_keywords(link, keywords):
            contained_links.append(link)
            keyword_found = True
    entry.append(pat)  # Titre du PAT
    entry.append(pat)  # Recherche effectuée
    entry.append(len(links))  # Nombre de liens obtenus
    entry.append(links)  # Liens obtenus
    entry.append(", ".join(keywords))  # Mots clés recherchés as string
    entry.append(", ".join(contained_links))  # Liens contenant mots clés as string
    entry.append(keyword_found)  # True or False indicating keyword found
    res.append(entry)
    print(f"{i+1} {pat} : {keyword_found}")

for entry in res:
    print(entry)

# Convert res list to DataFrame
df = pd.DataFrame(res, columns=["Titre du PAT", "Recherche effectuée", "Nb de liens obtenus", "Liens obtenus", "Mots clé recherché", "Mot clé trouvé", "Keyword Found"])

# Get the current timestamp
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# File names with timestamp
excel_file = f"data_{timestamp}.xlsx"
csv_file = f"data_{timestamp}.csv"

# Save DataFrame to Excel
df.to_excel(excel_file, index=False)

# Save DataFrame to CSV
df.to_csv(csv_file, index=False)

print("Data saved successfully.")

In [ ]:
# Get the current timestamp
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# File names with timestamp
excel_file = f"data_{timestamp}.xlsx"
csv_file = f"data_{timestamp}.csv"

# Save DataFrame to Excel
df.to_excel(excel_file, index=False)

# Save DataFrame to CSV
df.to_csv(csv_file, index=False)

print("Data saved successfully.")

In [ ]:
# Convert result list to DataFrame
df = pd.DataFrame(res, columns=["Titre du PAT", "Recherche effectuée", "Nb de liens obtenus", "Liens obtenus", "Mots clé recherché", "Mot clé trouvé", "Keyword Found"])